In [54]:
import pandas as pd

In [55]:
df = pd.read_csv("spotify_millsongdata.csv")

In [56]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [57]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [58]:
df.shape

(57650, 4)

In [59]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [60]:
df =df.sample(10000).drop('link', axis=1).reset_index(drop=True)

In [61]:
df.head(10)

,artist,song,text
0,Boney M.,Ma Baker,Freeze I'm ma Baker \r\nPut your hands in the...
1,Smiths,A Rush And A Push And The Land Is Ours,Oh hello \r\nI am the ghost of Troubled Joe ...
2,Dusty Springfield,Gotta Get Used To You,Who d'you think you're kidding? \r\nYou walk ...
3,Waylon Jennings,Are You Sure Hank Done It This Way,"Lord it's the same old tune, fiddle and guitar..."
4,Tiffany,Butterfly,"(J. Brooks, T. Feehan) \r\nYou should know th..."
5,ABBA,Kisses Of Fire,Lay your head on my chest so you hear every be...
6,John Prine,Clay Pigeons,"I'm goin' down to the Greyhound Station, gonna..."
7,Patti Smith,Soul Kitchen,"[Originally by The Doors] \r\n \r\nWell, the..."
8,Bob Marley,All Day All Night,"Wohoo, Said I'd never gonna give my love to no..."
9,Bon Jovi,In And Out Of Love,Young and wired \r\nSet to explode in the hea...


In [62]:
df['text'][0]

"Freeze I'm ma Baker  \r\nPut your hands in the air and give me all your money  \r\n  \r\nThis is the story of ma Baker the meanest cat from old Chicago town  \r\nShe was the meanest cat  \r\nIn old Chicago town  \r\nShe was the meanest cat  \r\nShe really mowed them down  \r\nShe had no heart at all  \r\nNo no no heart at all  \r\nShe was the meanest cat  \r\nFor she was really tough  \r\nShe left her husband flat  \r\nHe wasn't tough enough  \r\nShe took her boys along  \r\nCause they were mean and strong  \r\n  \r\nMa ma ma ma  \r\nMa Baker  \r\nShe taught her four sons  \r\nMa ma ma ma  \r\nMa Baker  \r\nTo handle there guns  \r\nMa ma ma ma  \r\nMa Baker  \r\nShe never could cry  \r\nMa ma ma ma  \r\nMa Baker  \r\nBut she knew how to die  \r\n  \r\nThey left a trail of crime  \r\nAcross the USA  \r\nAnd when one boy was killed  \r\nShe really made them pay  \r\nShe had no heart at all  \r\nNo no no heart at all  \r\n  \r\nMa ma ma ma  \r\nMa Baker  \r\nShe taught her four sons  \r

In [63]:
# df = df.sample(5000)

In [64]:
df.shape

(10000, 3)

Text Cleaning/ Text Preprocessing

In [65]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [66]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [67]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [70]:
similarity[0]

array([1.        , 0.00195269, 0.00102717, ..., 0.00721999, 0.00603789,
       0.00334151])

In [71]:
df[df['song'] == 'Crying Over You']

,artist,song,text
3338,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [72]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [73]:
recommendation('Crying Over You')

['My Cup',
 'Cry Baby Cry',
 'Crying',
 'Cry Cry Cry',
 "Don't Cry",
 "Don't Come Cryin' To Me - Previously Unreleased",
 "Cry, Cry Darlin'",
 'Cry Me A River',
 'Cry Baby',
 'Cry',
 'Cry Me A River',
 'My Heart Cries For You',
 "Don't Say Goodbye Again",
 'Cry A While',
 'When You Cry',
 'One Last Cry',
 'Cry Me A River',
 "Don't Cry Baby",
 'One Last Cry',
 'Fly Away']

In [74]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))